In [ ]:
import sys
sys.path.append(r"C:\Users\hjia9\Documents\GitHub\data-analysis")
sys.path.append(r"C:\Users\hjia9\Documents\GitHub\data-analysis\read")

import numpy as np
import pandas as pd
import os
import matplotlib as mpl
import matplotlib.pyplot as plt

import data_analysis_utils as utils
import calibrate_power as cbp
from read_scope_data import read_trc_data
from crds_tools import read_excel_sheets, get_sheet_names

%matplotlib qt

In [ ]:
path = r"C:\data\epfl\diagnostic-source\CRDS\scope-trc\solAnt"
ifn = r"C:\data\epfl\diagnostic-source\CRDS\solenoid\20230601_26mTorr_0.21lnmin_2.xlsx"
start = 113000
stop = 123000
pressure = 26

# Plot light signal and range of CW to take average for calibrating power later
file_ls = utils.get_files_in_folder(path, modified_date='2023-06-01', omit_keyword='water')
for file in file_ls:
    if "C3" in file and '-'+str(pressure)+'mT' in file:
        light, tarr = read_trc_data(file)
        plt.figure()
        plt.plot(tarr*1e3, light)
        plt.plot(tarr[start:stop]*1e3, light[start:stop])# plot range to take average
        plt.title(file)

In [ ]:
pwr_ls = cbp.get_power_solenoid(pressure, start, stop)
name_ls = get_sheet_names(ifn)

if False:
    raw_dat_arr_0 = read_excel_sheets(ifn, name_ls[-1], [0,1])
    raw_dat_arr_1 = read_excel_sheets(ifn, name_ls[-1], [11,12])

    plt.figure()
    plt.plot(raw_dat_arr_0[:,0], raw_dat_arr_0[:,1])
    plt.plot(raw_dat_arr_1[:,0], raw_dat_arr_1[:,1])

In [ ]:
dic = {}
for i, name in enumerate(name_ls):
    if i==0 or i==1:
       dic[name] = read_excel_sheets(ifn, name, [0,1])
    # if i==2:
    #     dic[name] = read_excel_sheets(ifn, name, [11,12])

print(dic.keys())

In [ ]:
converted_dic = {}
for i, key in enumerate(dic.keys()):
    print(key)
    data = dic[key]
    tarr_crds = data[:,0]
    sig_crds = data[:,1]
    plt.figure()
    plt.plot(tarr_crds, sig_crds)
    plt.title(pwr_ls[i,1])
    plt.xlabel('Time (s)')

    # signal baseline
    start = 10
    if i==0:
        stop = 20
    if i==1:
        stop = 18
    if i==2:
        stop = 20
    plt.scatter(tarr_crds[start:stop], sig_crds[start:stop], c='r')
    sig_base = np.average(sig_crds[start:stop])

    # signal active-glow equilibrium value
    start = 850
    stop = 950
    plt.scatter(tarr_crds[start:stop], sig_crds[start:stop], c='g')
    sig_val = np.average(sig_crds[start:stop])

    n = (sig_val-sig_base) / 5.8e-18 # convert CRDS to O- density

    converted_dic[str(int(pwr_ls[i,1]))+'W'] = n

    print(f"Density: {format(n, '.2e')}")

In [ ]:
'''
.npy file with the following format:
    [converted_dic, dic]
    converted_dic: dictionary with power as key and O- density as value
    dic: dictionary with sheet name as key and 2D array of time(s) and 1/c.tau(1/cm) from Excel as value
'''
npy_file_path = r"C:\data\epfl\diagnostic-source\CRDS\solenoid\{}mTorr.npy".format(pressure)
utils.save_to_npy([converted_dic,dic], npy_file_path)

In [ ]:
data = utils.read_from_npy(npy_file_path)
print(data[0].keys())
print(data[1].keys())
print(list(data[1].values())[0].shape)

In [ ]:
# Some C1 records antenna current, code commented out included as needed
ifn = r"C:\data\epfl\CRDS\scope trc\bdcageAnt\C1-300W-1mT-00000.trc"
Iant, tarr = read_trc_data(ifn)
Iant *= 8 # Pearson 8A/V

if "C4" in file:
    Vtrig, tarr = read_trc_data(file)
    plt.figure(2)
    plt.plot(tarr*1e3, Vtrig, label = str(power) + "W")
    plt.ylabel('TTL')
    plt.xlabel('time (ms)')
    plt.legend()

In [ ]:
fig, ax = plt.subplots(nrows=3, sharex=True)
fig.suptitle('5mTorr/0.67Pa 580W')

ax[0].plot(tarr*1e3, light)
ax[0].set_ylabel('Light')

ax[1].plot(tarr*1e3, Vtrig)
ax[1].set_ylabel('TTL')

tarr_fake = tarr_crds + tarr_crds[-1]
tarr_crds_mod = np.hstack((tarr_crds, tarr_fake)) #+tarr_crds[100]
sig_crds_mod = np.tile(sig_crds , 2) #numpy.roll(sig_crds,-100)

ax[2].plot(tarr_crds_mod*1e3, sig_crds_mod)
ax[2].set_ylabel('crds signal')
ax[2].set_xlabel('time (ms)')